# Use data_extraction.py here

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, importlib.util, sys, glob

MODULE = "/content/drive/MyDrive/Colab Notebooks/data_extraction.py"
print("file exists?", os.path.isfile(MODULE))


spec = importlib.util.spec_from_file_location("data_extraction", MODULE)
data = importlib.util.module_from_spec(spec)
spec.loader.exec_module(data)

Mounted at /content/drive
file exists? True


# Download data for one company

In [ ]:
import pandas as pd

company = pd.Series({
    'CIK': '0001079114',
    'Company Name': 'MGREENLIGHT CAPITAL INC'
})

df = data.download_13f_in_date_range(
    start_date="2020-01-01",
    end_date="2025-06-30",
    data=company,
    save=False
)
print(df.head())

In [4]:
df.columns

Index(['Company Name', 'CIK', 'nameOfIssuer', 'cusip', 'value', 'shares',
       'shareType', 'putCall', 'investmentDiscretion', 'votingAuthoritySole',
       'votingAuthorityShared', 'votingAuthorityNone', 'filing_date',
       'filing_url'],
      dtype='object')

# Try to get map: cusip->sector

In [5]:
!pip -q install yfinance rapidfuzz

import re, time, requests, pandas as pd
import yfinance as yf
from rapidfuzz import fuzz, process

HEADERS = {"User-Agent": "Your Name you@example.com"}

# 1) Download the SEC company list (ticker + company name + CIK)
def load_sec_tickers():
    url = "https://www.sec.gov/files/company_tickers.json"
    js = requests.get(url, headers=HEADERS, timeout=20).json()
    rows = []
    for v in js.values():
        rows.append({
            "cik": str(v["cik_str"]).zfill(10),
            "ticker": v["ticker"],

            "name": v["title"],
        })
    return pd.DataFrame(rows)

# Name normalization for easier matching
def norm_name(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9 ]+", " ", str(s)).upper()
    s = re.sub(r"\b(CORP(ORATION)?|INC(ORPORATED)?|LTD|LLC|LP|PLC|CO|COMPANY|HLDGS?|HOLDINGS?)\b", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

sec = load_sec_tickers()
sec["name_norm"] = sec["name"].apply(norm_name)

# 2) Extract the unique issuer (cusip + nameOfIssuer) from 13F df
# -- Using df here --
issuers = df[["cusip","nameOfIssuer"]].drop_duplicates().copy()
issuers["name_norm"] = issuers["nameOfIssuer"].apply(norm_name)

# First do an exact merge of the standard name
m = issuers.merge(sec[["ticker","name_norm"]], on="name_norm", how="left")

# For those that are not matched, use fuzzy matching to make up for them (the threshold is adjustable)
unmatched = m[m["ticker"].isna()] # so here unmatched is a new df only containing the rows where ticker's value is NA
if not unmatched.empty:
    choices = sec[["name_norm","ticker"]].drop_duplicates()
    idx_map = {}
    for i, row in unmatched.iterrows():
        q = row["name_norm"]
        match = process.extractOne(q, choices["name_norm"], scorer=fuzz.token_sort_ratio)
        if match and match[1] >= 90:  # Similarity threshold
            hit = choices[choices["name_norm"]==match[0]].iloc[0]
            idx_map[i] = hit["ticker"]
    for i, t in idx_map.items():
        m.at[i, "ticker"] = t

# 3) Use yfinance to pull the sector and map it to the GICS 11
def yf_sector(tk):
    try:
        info = yf.Ticker(tk).get_info()  # may be slow
        return info.get("sector")
    except Exception:
        return None

print("Fetching sectors from Yahoo Finance …")
m["sector_raw"] = None
for i, tk in m["ticker"].dropna().items():
    m.at[i, "sector_raw"] = yf_sector(tk)
    time.sleep(0.2)  # Be gentle and avoid being restricted

# Mapping to GICS 11 (handling synonyms)
map_to_gics = {
    "Energy":"Energy",
    "Materials":"Materials",
    "Industrials":"Industrials",
    "Consumer Cyclical":"Consumer Discretionary",
    "Consumer Defensive":"Consumer Staples",
    "Healthcare":"Health Care",
    "Financial Services":"Financials",
    "Technology":"Information Technology",
    "Communication Services":"Communication Services",
    "Utilities":"Utilities",
    "Real Estate":"Real Estate",
}
m["sector"] = m["sector_raw"].map(map_to_gics).fillna(m["sector_raw"])

# Get sector_map (cusip -> GICS industry), when there are many to one, the first or majority
sector_map = (m.dropna(subset=["sector"])
                .groupby("cusip")["sector"]
                .agg(lambda s: s.value_counts().idxmax())
                .reset_index())
print("sector_map rows:", len(sector_map))
sector_map.head()

Fetching sectors from Yahoo Finance …
sector_map rows: 74


,cusip,sector
0,00187Y100,Industrials
1,00217D100,Information Technology
2,00751Y106,Consumer Discretionary
3,02156K103,Communication Services
4,02209S103,Consumer Staples


# Build traj

In [19]:
# ---------- deps ----------
import numpy as np
import pandas as pd
import torch
import yfinance as yf
import re

# ---------- constants ----------
GICS_ORDER = [                  # Fixed order of the 11 GICS sectors (as column order/dimension)
    "Energy","Materials","Industrials","Consumer Discretionary","Consumer Staples",
    "Health Care","Financials","Information Technology","Communication Services",
    "Utilities","Real Estate"
]
SECTOR_TICKERS = {              # Common ETF codes corresponding to 11 industries (used to approximate industry returns)
    "Energy":"XLE","Materials":"XLB","Industrials":"XLI","Consumer Discretionary":"XLY",
    "Consumer Staples":"XLP","Health Care":"XLV","Financials":"XLF","Information Technology":"XLK",
    "Communication Services":"XLC","Utilities":"XLU","Real Estate":"XLRE",
}

# ---------- helpers ----------

# Normalize the input (number/string) into a legal industry name
def _to_gics_name(s):
    if pd.isna(s): return None  # Missing values return None directly (indicating that they cannot be mapped)
    if isinstance(s,(int,np.integer)) and 0<=int(s)<11:
        return GICS_ORDER[int(s)]  # If it is a number between 0 and 10, the subscript is the industry name
    s = re.sub(r'\s+',' ',str(s)).strip().lower()  # Unify uppercase and lowercase letters and spaces
    for name in GICS_ORDER:
        if s == name.lower():   # Returns an exact match to the standard industry name (lowercase)
            return name
    return None

# Find the latest date <= target in time index idx
def _nearest(idx, target):
    pos = idx.searchsorted(pd.to_datetime(target), side="right") - 1  # Binary search positioning
    pos = max(0, min(pos, len(idx)-1))  # Boundary protection, guaranteed to be in [0, len-1]
    return idx[pos]             # Return the most recent valid date

# Compatible with yfinance
def _pick_close(df):
    if isinstance(df.columns, pd.MultiIndex):
        lvl0 = df.columns.get_level_values(0)
        key = "Adj Close" if "Adj Close" in lvl0 else "Close"  # Priority adjustment closing
        return df[key]
    else:
        cols = df.columns.tolist()
        key = "Adj Close" if "Adj Close" in cols else "Close"
        return df[key]

# Download price series for industry ETFs and SPY
def _fetch_sector_spy_prices(periods, pad_days=7):
    start = pd.to_datetime(min(periods)) - pd.Timedelta(days=pad_days)  # Start a little early
    end   = pd.to_datetime(max(periods)) + pd.Timedelta(days=pad_days)  # Delay the end

    raw = yf.download(" ".join(SECTOR_TICKERS.values()),
                      start=start, end=end, auto_adjust=False, progress=False)  # Pull industry ETFs
    px = _pick_close(raw)                     # Get the adjusted/closing price subtable
    if isinstance(px, pd.Series): px = px.to_frame()  # Convert a single column into a two-dimensional table for easy subsequent processing
    px = (px.rename(columns={v:k for k,v in SECTOR_TICKERS.items()})     # Column name changed from ETF to industry name
            .reindex(columns=GICS_ORDER)                                 # Sort columns by fixed industry order
            .sort_index().ffill().bfill())                               # Sort by date and fill in missing values

    spy_raw = yf.download("SPY", start=start, end=end,
                          auto_adjust=False, progress=False)              # Download SPY prices
    spy = _pick_close(spy_raw).sort_index().ffill().bfill()              # Same
    return px, spy                   # Industry Price List & SPY Price Series

def _sector_simple_returns_between_filings(periods, sector_px, spy_px=None, excess=True):
    rows = []                       # Save the 11-dimensional simple benefits of each interval
    for t in range(len(periods)-1): # Traverse adjacent disclosure periods (t -> t+1)
        t0, t1 = _nearest(sector_px.index, periods[t]), _nearest(sector_px.index, periods[t+1])  # Align to trading day
        s0, s1 = sector_px.loc[t0], sector_px.loc[t1]  # Take the industry prices at both ends of the interval
        r = (s1/s0 - 1.0).astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)  # Calculate simple benefits and clean up
        if excess and spy_px is not None:            # If you want excess returns (relative to SPY)
            b0 = float(spy_px.loc[_nearest(spy_px.index, periods[t])])      # Range starting point SPY
            b1 = float(spy_px.loc[_nearest(spy_px.index, periods[t+1])])    # End point of the interval SPY
            r = r - (b1/b0 - 1.0)                 # Industry Return - SPY Return = Excess
        rows.append(r.values.astype(np.float32))
    return np.stack(rows, axis=0)  # [T-1,11]

def _estimate_ar1(R_obs, ridge=1e-6):  # Estimate industry-by-industry AR(1) parameters using observed returns
    """Industry by industry AR(1): r_t = phi * r_{t-1} + eps"""
    if R_obs.shape[0] < 3:              # Give a gentle default value when there are too few samples
        phi = np.zeros(11, dtype=np.float32)
        Sigma = 1e-5 * np.eye(11, dtype=np.float32)  # The residual covariance takes a very small diagonal matrix
        return phi, Sigma
    Y, X = R_obs[1:], R_obs[:-1]        # Y is the current return, X is the return lagged by one period
    phi = np.zeros(11, dtype=np.float32); resid = np.zeros_like(Y)  # Pre-allocation coefficients and residuals
    for i in range(11):                 # Separate regression for each industry
        num = float(np.dot(Y[:,i], X[:,i])); den = float(np.dot(X[:,i], X[:,i]) + ridge)  # Least squares (with ridge term)
        phi[i] = num/den                # Slope coefficient phi_i
        resid[:,i] = Y[:,i] - phi[i]*X[:,i]   # Calculate the residual sequence
    Sigma = np.cov(resid, rowvar=False)       # Covariance matrix of residuals
    Sigma = 0.5*Sigma + 0.5*np.diag(np.diag(Sigma)) + 1e-8*np.eye(11)  # Stabilization (shrink to the diagonal)
    return phi, Sigma                  # Return AR(1) coefficients and residual covariances




# ---------- main: df -> traj (AR1) ----------
def build_sector_traj_from_df_AR1(df_raw, sector_map, value_unit=1000.0, device="cpu",
                                  min_shares=1e-6, use_excess=True, seed=42):
    """
    Return traj：
      x_t      [T,11] Industry USD holdings
      r_t      [T-1,11] Realized simple industry returns (here "real raw returns", used for calculating u_t)
      bar_r_t  [T-1,11] Expected return bar_r_t = Phi * r_{t-1} (estimated based on the sequence specified by use_excess)
      Sigma_r  [11,11]  Residual covariance estimation
      u_t      [T-1,11] Net USD trading volume (calculate according to MDP PDF)
      C_t      [T-1,1]  net cash flow
      B_t      [T-1,1]  Benchmark
      cash_t   [T,1]    Cash Processing (USD)
    """
    df = df_raw.copy()
    # check columns
    need = {'cusip','value','shares','filing_date'}
    if not need.issubset(df.columns):
        raise ValueError(f"df_raw need columns: {need}")
    df['value']  = pd.to_numeric(df['value'],  errors='coerce').fillna(0.0)          # convert to number
    df['shares'] = pd.to_numeric(df['shares'], errors='coerce').fillna(0.0)          # convert to number
    df['filing_date'] = pd.to_datetime(df['filing_date'])                            # convert to datetime

    # Sector Map to 11 Industries
    m = dict(sector_map[['cusip','sector']].values) if isinstance(sector_map,pd.DataFrame) else dict(sector_map)
    df['sector'] = df['cusip'].map(m).map(_to_gics_name)                              # Get standardized industry name based on mapping
    df = df.dropna(subset=['sector']).copy()                                          # Discard rows that cannot be mapped to an industry(can be adjusted)

    periods = sorted(df['filing_date'].unique().tolist())                             # Take all disclosure periods and sort them
    if len(periods) < 2:                                                              # Need at least two periods
        raise ValueError("At least two issues of 13F are required to construct the trajectory.")

    # x_t
    val_sector = (df.groupby(['filing_date','sector'])['value']                       # Aggregate value by (period, industry) (unit: thousands of US dollars)
                    .sum().unstack('sector')                                          # Transferred industries
                    .reindex(index=periods, columns=GICS_ORDER, fill_value=0.0)       # Align rows and columns to a fixed order and fill with zeros
                    .sort_index())                                                    # Sort by date
    X_dollar = (val_sector * float(value_unit)).astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)  # Multiply units to convert to dollars and clean up exceptions
    X = torch.tensor(X_dollar.values, dtype=torch.float32, device=device)             # [T,11] Industry USD holdings tensor
    AUM = X_dollar.sum(axis=1).astype(float)                                          # [T] Total assets per period (USD)

    # ------- Separate "real raw returns for transfers" and "series for estimates/expectations" -------
    sector_px, spy_px = _fetch_sector_spy_prices(periods)                             # Download sector ETF and SPY prices

    # (1) r_t_real_raw：True "raw" industry returns (not excess), driving transfers and resolving u_t
    R_real_np = _sector_simple_returns_between_filings(                               # Calculate the raw industry returns for adjacent disclosure periods
        periods, sector_px[GICS_ORDER], spy_px, excess=False                          # excess=False means no SPY reduction
    )
    R_real = torch.tensor(R_real_np, dtype=torch.float32, device=device)              # Convert to tensor [T-1,11] as realized r_t

    # (2) R_for_model：Series to use for estimating AR(1) (optional: excess or original, controlled by use_excess)
    R_for_model_np = _sector_simple_returns_between_filings(                           # The second set of return series is only used for modeling
        periods, sector_px[GICS_ORDER], spy_px, excess=use_excess                     # True=Excess, False=Original
    )
    phi, Sigma = _estimate_ar1(R_for_model_np)                                        # Use this sequence to estimate the AR(1) coefficient and residual covariance
    phi_t = torch.tensor(phi, dtype=torch.float32, device=device)                     # Coefficients converted to tensors (11,)

    # bar_r_t = Phi * r_{t-1}
    bar_R = torch.zeros_like(R_real)
    bar_R[0] = 0.0                                                                    # The first has no previous value, so set to 0
    bar_R[1:] = torch.tensor(R_for_model_np[:-1], dtype=torch.float32, device=device) * phi_t  # Sector by sector Φ r_{t-1}

    Sigma_t = torch.tensor(Sigma, dtype=torch.float32, device=device)                 # Covariance Matrix Tensorization [11,11]

    # ------- Use "real original income" to reverse solve u_t -------
    den = (1.0 + R_real).clamp_min(1e-6)
    U = (X[1:] / den) - X[:-1]
    C = U.sum(dim=1, keepdim=True)
    B = torch.zeros((len(periods)-1, 1), dtype=torch.float32, device=device)          # Benchmark[T-1,1]

    # Cash
    cash_usd = [torch.tensor([0.0], dtype=torch.float32, device=device)]              # Initial cash (USD), set to 0 here
    cost_rate = 1e-3                                                                   # Transaction fee rate (e.g. 0.1% = 10bps)
    for t in range(len(periods)-1):                                                    # Update cash balance period by period
        traded_dollars = torch.linalg.vector_norm(U[t], ord=1)                         # Approximate transaction amount: sum of L1 norm
        trade_cost_usd = cost_rate * traded_dollars                                    # Transaction costs (USD)
        cash_next_usd = torch.clamp(cash_usd[-1] + C[t] - trade_cost_usd, min=0.0)     # Cash = Cash + Net Inflow - Cost, and is not negative
        cash_usd.append(cash_next_usd)
    CASH_USD = torch.stack(cash_usd, dim=0)                                            # [T,1]

    # Clean up(reset NaN/Inf to zero to ensure stability of subsequent training)
    X        = torch.nan_to_num(X,        nan=0.0, posinf=0.0, neginf=0.0)
    R_real   = torch.nan_to_num(R_real,   nan=0.0, posinf=0.0, neginf=0.0)
    bar_R    = torch.nan_to_num(bar_R,    nan=0.0, posinf=0.0, neginf=0.0)
    U        = torch.nan_to_num(U,        nan=0.0, posinf=0.0, neginf=0.0)
    C        = torch.nan_to_num(C,        nan=0.0, posinf=0.0, neginf=0.0)
    CASH_USD = torch.nan_to_num(CASH_USD, nan=0.0, posinf=0.0, neginf=0.0)

    return {
        "x_t": X,                                                                      # [T,11]
        "r_t": R_real,                                                                 # [T-1,11]
        "bar_r_t": bar_R,                                                              #（Φ r_{t-1})[T-1,11]
        "Sigma_r": Sigma_t,                                                            # [11,11]
        "u_t": U, "B_t": B, "C_t": C, "cash_t": CASH_USD,                               # action/benchmark/net_cash_flow/cash state(start from 0)
        "periods": periods, "sectors": GICS_ORDER,                                     # time point and industry order
        "ar1": {"phi": torch.tensor(phi)}                                              # AR(1) coefficient
    }


# ---------- small utility: traj -> states ----------
def build_states_from_traj(traj):  # Split traj into a list of "step-by-step states" (for easier feeding to the model)
    """
    Returns a list of length T-1, each element is a state s_t:
      {'x': [11], 'bar_r': [11], 'Sigma_r': [11,11], 'C': [1]}
    """
    X, barR, C, Sigma = traj["x_t"], traj["bar_r_t"], traj["C_t"], traj["Sigma_r"]  # Extract the required fields
    T = X.shape[0]                # Total number of periods T
    states = []                   # The list of states that will be returned
    for t in range(T-1):          # Only until T-1 (each state is used in the interval t->t+1)
        states.append({"x": X[t], "bar_r": barR[t], "Sigma_r": Sigma, "C": C[t]})  # Pack the features of each step into a dictionary
    return states                 # Return a chronological sequence of states


In [ ]:
# 1) First construct traj from df (single company) and sector_map (cusip -> GICS industry)
traj = build_sector_traj_from_df_AR1(
    df_raw=df,
    sector_map=sector_map,      # DataFrame or dict: {'cusip':'Information Technology', ...}
    value_unit=1000.0,          # If your value is already "USD", change it to 1.0(In 13f it's 1000 USD)
    use_excess=True,            # Sector Returns - Benchmark (SPY)
    seed=42
)

# size
for k in ["x_t","r_t","bar_r_t","Sigma_r","u_t","C_t","cash_t"]:
    print(k, traj[k].shape if hasattr(traj[k], "shape") else type(traj[k]))

# 2) states：
states = build_states_from_traj(traj)
print(len(states), states[0]["x"].shape, states[0]["bar_r"].shape)

In [ ]:
states[0]